# Advanced Topics in Molecular Dynamics
Gábor Csányi (Engineering Laboratory, Cambridge, UK)

# jupyter 

ju(lia) py(thon) ter(minal)



## Lecture 1: Tools

* Jupyter notebooks: compute and plotting environment
* Julia language: best bits of Matlab and Python, just-in-time compiled http://julialang.org
 * (Python also possible: interfaces to atomistic tools are via Python anyway)
* ASE (Atomic Simulation Environment), a Python package to manage atomic configurations
 * Homepage: https://wiki.fysik.dtu.dk/ase/
 * James Kermode's fork: https://gitlab.com/jameskermode/ase
* QUIP (QUantum mechanics and Interatomic Potentials) https://github.com/libAtoms/QUIP
 * many potentials and their combinations, including wrappers to other codes, Python interface
* LAMMPS (Large-scale Atomic/Molecular Massively Parallel Simulator) http://lammps.sandia.gov

# Lecture 2: Molecular Dynamics

* Discretisation, Langevin thermostats
* Sampling error, error bars
* The hydrogen bond in the water dimer
* Thermal expansion of the water hexamer


# Lecture 3: Function fitting and interpolation

* Kernel fitting (basis functions, regularisation)
* Gaussian process (variance, noise tolerance)
* Free energy suface reconstruction: fitting a function from noisy derivatives
 

# Lecture 4: Fitting interatomic potentials

* Notions of locality in quantum mechanics
* Body expansion (materials systems, water dimer)
* Fitting to linear combinations of data: total energies and forces
* Example: bulk silicon, tungsten
* Fitting the difference between potential energy surfaces: helium in tungsten  


# Lecture 5: Atomistic and molecular descriptors

* An incomplete desriptor: unordered interatomic distances, reconstruction test
* Smooth Overlap of Atomic Positions (SOAP)
* SOAP as an order parameter: identify structures and defects
* Molecular and period structures: the reMATCH kernel
* Map of chemical space


# Lecture 6: Thermodynamics from first principles

* Partition function, response functions
* First order phase transitions
* Nested sampling
* Lennard-Jones clusters, condensed phase diagram
* Comparison to parallel tempering, Wang-Landau


In [ ]:
include("mdlecturesrc.jl")

In [ ]:
miframe("http://julialang.org")

In [ ]:
miframe("https://wiki.fysik.dtu.dk/ase/")

In [ ]:
miframe("http://lammps.sandia.gov")

## Create a molecule

In [ ]:
h2o = quippy.Atoms(n=3, lattice=[10 0 0 ; 0 10 0 ; 0 0 10], numbers=[8,1,1],
                    positions=[0 0 0; 0.96 0 0 ; -0.23 0.93 0]);

In [ ]:
imolecule_draw(h2o)

## Parametrisation of the water monomer potential enegy surface by Partridge and Schwenke

Harry Partridge & David W Schwenke, The Journal of Chemical Physics, __106__ 4618–4639 (1997)

_The determination of an accurate isotope dependent potential energy surface for water from extensive ab initio calculations and experimental data._



In [ ]:
ps = quippy.Potential("IP PartridgeSchwenke force_using_fd=T", 
                        param_filename="quip_params.xml");

In [ ]:
ps[:calc](h2o, args_str="energy"); println(h2o[:energy])

In [ ]:
x,y = grids.meshgrid(-1.0:0.05:0.4, [(-1.25:0.05:-0.4) ;(0.4:0.05:1.25)])
h2o_ps_energy = zeros(x)
for i=1:length(x)
    h2o[:set_positions]([0 0 0; 0.96 0 0 ; x[i] y[i] 0])
    ps[:calc](h2o, args_str="energy"); h2o_ps_energy[i] = h2o[:energy]
end

In [ ]:
figure(figsize=(7,5))
contourf(x, y, h2o_ps_energy, 0:0.1:1, cmap="winter"); title("H2O Potential energy surface (Partridge-Schwenke)")
contour(x, y, h2o_ps_energy, [0.025], colors="w", linestyles="dashed")
axis([-1, 1.2, -1.3, 1.3])
patch_h2o()
close(gcf())

## Fitting quadratic bond and angle springs

$$
E_\mathrm{quad} = \alpha_0 + \alpha_1 \theta + \alpha_2 \theta^2 + \sum_{i \in \{1,2\}} \beta_0 + \beta_1 r_{\mathrm{OH}_i} + \beta_2 r_{\mathrm{OH}_i}^2
$$

Least squares fit:

$$
\alpha^*, \beta^* = \mathrm{arg}\min_{\alpha,\beta} \sum_{n\in\mathrm{configs}} \left|E_\mathrm{quad}(n | \alpha,\beta) - E_\mathrm{PS}(n)\right|^2
$$

Coefficients in $E_\mathrm{quad}$ are _linear_, so fit can be accomplished by linear algebra, e.g. by choosing configurations in which only one coordinate changes, i.e. one of the bond lengths, or the bond angle

#### Compute the energy as a function of O-H bond distance

In [ ]:
r = 0.88:0.01:1.05
Er = zeros(r)
for i=1:length(r)
    h2o[:set_positions]([0 0 0; r[i] 0 0 ; 0.96*cos(105/180*π) 0.96*sin(105/180*π) 0])
    ps[:calc](h2o, args_str="energy"); Er[i] = h2o[:energy]
end

#### Compute the energy as a function of H-O-H bond angle

In [ ]:
θ = (90.0:120.0)/180*π
Eθ = zeros(θ)
for i=1:length(θ)
    h2o[:set_positions]([0 0 0; 0.96 0 0 ; 0.96*cos(θ[i]) 0.96*sin(θ[i]) 0])
    ps[:calc](h2o, args_str="energy"); Eθ[i] = h2o[:energy]
end

In [ ]:
figure(figsize=(8,4))
subplot(121)
plot(r, Er, "o-")
plot([r[1],r[end]], [0.025,0.025], "k:")
xlabel("O-H distance / A"); ylabel("Energy / eV")

subplot(122)
plot(θ*180/π, Eθ, "o-")
plot([θ[1],θ[end]]*180/π, [0.025,0.025], "k:")
xlabel("H-O-H angle "); ylabel("Energy / eV")

tight_layout()
close(gcf())

#### compute coefficients 

$$
\beta^T A \approx E
$$

$$
\left[\begin{matrix}
\beta_0\\
\beta_1\\
\beta_2
\end{matrix}\right]^T
\left[\begin{matrix}
1 & 1 & \ldots & 1\\
r_1(1) & r_1(2) & \ldots & r_1(n)\\
r_1^2(1) & r_1^2(2) & \ldots & r_1^2(n)\\
\end{matrix}\right]
\approx
\left[\begin{matrix}
E(1) & E(2) & \ldots & E(n)
\end{matrix}\right]
$$

$$
\Rightarrow\qquad
\beta^T =  E A^{+}
$$

where $A^{+}$ is the pseudoinverse, such that $A A^{+} = I$, given by $A^{+} = A(AA)^{-1}$

#### Fit quadratic

In [ ]:
rr = r[4:14]
A = [ones(length(rr))' ;  rr' ; rr'.^2 ]
bond = A' \ Er[4:14]
θθ = θ[12:20]
A = [ones(length(θθ))' ;  θθ' ; θθ'.^2 ]
angle = A' \ Eθ[12:20]

In [ ]:
figure(figsize=(8,4))
subplot(121)
plot(r, Er, "o-")
plot(r, bond[1]+bond[2]*r+bond[3]*r.^2, "r-")
plot([r[1],r[end]], [0.025,0.025], "k:")
xlabel("O-H distance / A"); ylabel("Energy / eV")

subplot(122)
plot(θ*180/π, Eθ, "o-")
plot(θ*180/π, angle[1]+angle[2]*θ+angle[3]*θ.^2, "r-")
plot([θ[1],θ[end]]*180/π, [0.025,0.025], "k:")
xlabel("H-O-H angle "); ylabel("Energy / eV")

tight_layout()
close(gcf())

θ0 = (-angle[2]/(2*angle[3]))
r0 = (-bond[2]/(2*bond[3]))
println(" ")
print("Bond coeffs: "); println(bond')
print("Angle coeffs: "); println(angle')
println("θ0 = $(θ0*180/π)")
println("r0 = $r0")

In [ ]:
miframe("http://www1.lsbu.ac.uk/water/water_models.html")

## Create a quadratic approximation energy model


In [ ]:
function h2o_quad(p)
    bond = [24.1648  -50.3605    26.2387   ]     # coefficients for bond A angstroms
    angle = [7.207    -0.137997   0.000660598]   # coefficient for angle in degrees
    
    r12 = norm(p[4:6]-p[1:3])
    r13 = norm(p[7:9]-p[1:3])
    θ = acos(sum((p[4:6]-p[1:3]).*(p[7:9]-p[1:3]))/(r12*r13))/π*180.0
    E = bond[1]+bond[2]*r12+bond[3]*r12*r12 + bond[1]+bond[2]*r13+bond[3]*r13*r13+
        angle[1]+angle[2]*θ+angle[3]*θ*θ
end


In [ ]:
xx,yy = grids.meshgrid(-0.45:0.02:-0.05, 0.85:0.01:1.0)
r1 = [0.93,0.94,0.96,0.98,0.985]
h2o_quad_energy_xxyy = zeros(size(xx)..., size(r1)...)
h2o_ps_energy_xxyy = zeros(size(xx)..., size(r1)...)

for j=1:length(r1)
    for i=1:length(xx)
        h2o[:set_positions]([0 0 0; r1[j] 0 0 ; xx[i] yy[i] 0])
        h2o_quad_energy_xxyy[ind2sub(size(xx),i)...,j] = h2o_quad(vec(h2o[:get_positions]()'))
        ps[:calc](h2o, args_str="energy"); h2o_ps_energy_xxyy[ind2sub(size(xx),i)...,j] = h2o[:energy]
    end
end

In [ ]:
f = figure()
@manipulate for j=1:size(h2o_ps_energy_xxyy,3) withfig(f) do
        contour(xx, yy, h2o_ps_energy_xxyy[:,:,j], [0, 0.015, 0.025, 0.035, 0.05, 0.1, 0.2, 0.4, 0.8, 1.6], colors="k")
        contour(xx, yy, h2o_quad_energy_xxyy[:,:,j], [0, 0.015, 0.025, 0.035, 0.05, 0.1, 0.2, 0.4, 0.8, 1.6], colors="r")
        contour(xx, yy, h2o_ps_energy_xxyy[:,:,j], [0.025], colors="k", linewidths=3) 
        contour(xx, yy, h2o_quad_energy_xxyy[:,:,j], [0.025], colors="r", linewidths=3) 
        axis([-0.45, -0.05, 0.85, 1.0])
        #axis("equal")
        title("H2O Potential energy surface |r1| = $(r1[j])")
        #legend()
        tight_layout()
    end
end


## Geometry optimisation

Finding the configuration with the lowest energy is a standard task of molecular modelling. 

Julia provides the Optim package

In [ ]:
using Optim

In [ ]:
#Optim.optimize()

In [ ]:
res = optimize(h2o_quad, vec(h2o[:get_positions]()'), ftol=1e-8)

#### Optimum geometry

In [ ]:
res.minimum

In [ ]:
p = reshape(res.minimum, (3,3))'

In [ ]:
norm(p[2,:]-p[1,:]), norm(p[3,:]-p[1,:])

## Multiple molecules

Add Coulomb, dispersion (Van der Waals) attraction and Pauli repulsion terms

$$
E = \sum_{i \in \mathrm{H}_2\mathrm{O}} E_\mathrm{quad}(i) + \sum_{ij\in\mathrm{O,H}} \frac{q_i q_j}{|r_{ij}|} + \sum_{i,j\in\mathrm{O}} V_\mathrm{LJ}(r_{ij})
$$

Dispersion attraction and Pauli repulsion are described by the Lennard-Jones potential
$$
V_\mathrm{LJ} = 4 \varepsilon \left[ \left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6 \right]
$$

In [ ]:
function tip3p_energy{T}(p::Vector{T})

    # input vector of positions in this orderL Ox Oy Oz Hx Hy Hz Hx Hy Hz Ox Oy Oz...
    
    qO = -0.8340
    qH = 0.4170
    σ = 3.15061       # in Angstrom
    ε = 0.6364 * 0.01 # conversion from kJ/mol to eV
    bohr = 0.529177249 # 1 Bohr in Angstrom
    hartree = 27.2114  # in eV
    
    E = 0.0
    
    for i=1:length(p)÷9                          # loop over oxygens (i.e. H2O monomers)
        E += h2o_quad(p[(i-1)*9+1:(i-1)*9+9])    # monomer energy
        for j=i+1:length(p)÷9                    # loop over other oxygens 
            O1 = (i-1)*9+1; O2 = (j-1)*9+1
            rij = norm(p[O1:O1+2]-p[O2:O2+2])
            E += 4.0*ε*((σ/rij)^12-(σ/rij)^6)     # LJ term
            # Electrostatics
            Es = 0.0
            Es += qO*qO/rij
            Es += qO*qH*(1/norm(p[O1:O1+2]-p[O2+3:O2+5])
                +1/norm(p[O1:O1+2]-p[O2+6:O2+8])
                +1/norm(p[O1+3:O1+5]-p[O2:O2+2])
                +1/norm(p[O1+6:O1+8]-p[O2:O2+2]))
            Es += qH*qH*(1/norm(p[O1+3:O1+5]-p[O2+3:O2+5])
                +1/norm(p[O1+3:O1+5]-p[O2+6:O2+8])
                +1/norm(p[O1+6:O1+8]-p[O2+3:O2+5])
                +1/norm(p[O1+6:O1+8]-p[O2+6:O2+8]))  
            E += Es*hartree*bohr
        end
    end
    E
end

## Water dimer

In [ ]:
h4o2 = quippy.Atoms(n=6, lattice=[10 0 0 ; 0 10 0 ; 0 0 10], numbers=repeat([8,1,1],outer=[2]),
                    positions=[ h2o_p0 ;  h2o_p0 +  repeat([3.0 0 0], outer=[3,1])]);
imolecule_draw(h4o2)

### Obtaining gradients

Many algorithms, including most geometry optimisation methods and molecular dynamics require the gradient of the potential. Notable exceptions:

* Nelder Mead and other gradient-free optimisation methods (but its much less efficient than ones based on gradients)
* Monte Carlo simulations (although very efficient moves can be generated by including gradient information) 

Options for computing the gradient of a function of _N_ variables: 

* Finite difference (e.g. Ridder's method), cost: O(_N_), optimum balance of roundoff error and truncation error 
* Hand code the gradient function, cost: O(1), but time consuming and prone to bugs
* _Automatic differentiation_: Julia is a language with full introspection, and has several such packages. 

In [ ]:
tip3p_grad = ForwardDiff.gradient(tip3p)

In [ ]:
h4o2[:rattle](0.1)
res = optimize(tip3p.potential, (p,s)->s[:] = tip3p.gradient(p), vec(h4o2[:get_positions]()'), ftol=1e-4, grtol=1e-2)

In [ ]:
h4o2[:set_positions](reshape(res.minimum, (3,6))')
imolecule_draw(h4o2)

### Dissociation curve

In [ ]:
r = 2.3:0.05:8.0
p0 = h4o2[:get_positions]()
p = copy(p0)
dOO = p0[4,:]- p0[1,:] 
Edim_tip3p = zeros(r)
Edim_fx = zeros(r)
for i=1:length(r)
    p[4:6,:] = p0[4:6,:] + repeat((r[i]/norm(dOO)-1.0)*dOO, outer=[3,1])
    Edim_tip3p[i] = tip3p.potential(vec(p'))
    h4o2[:set_positions](p); fx[:calc](h4o2, args_str="energy"); Edim_fx[i] = h4o2[:energy]
end

In [ ]:
figure()
plot(r, Edim_tip3p, "b-", label="TIP3P")
plot(r, Edim_fx, "g-", label="FX")
grid()
legend()
xlabel(L"r_\mathrm{OO} / \AA"); ylabel("Energy / eV")
close(gcf())

### Interaction of water molecules: TTM3F polarisable force field by Fanourgakis and Xantheas

George S. Fanourgakis and Sotiris S. Xantheas, Journal of Chemical Physics __128__, 074506 (2008)

_Development of transferable interaction potentials for water. V. Extension of the flexible, polarizable, Thole-type model potential (TTM3-F, v. 3.0) to describe the vibrational spectra of water clusters and liquid water_


In [ ]:
fx = quippy.Potential("IP FX force_using_fd=T virial_using_fd=T", param_filename="quip_params.xml");

## Create a water hexamer

In [ ]:
h12o6 = make_h12o6()
imolecule_draw(h12o6)

### Optimise its geometry

In [ ]:
h12o6[:rattle](0.1)
res = optimize(tip3p.potential, (p,s)->s[:]=tip3p.gradient(p), vec(h12o6[:get_positions]()'),
        ftol=1e-4, grtol=1e-4)

In [ ]:
h12o6[:set_positions](reshape(res.minimum, (3,18))')
imolecule_draw(h12o6)

### Geometry optimisation via QUIP

In [ ]:
fx[:minim](h12o6, "cg", 1e-1, 12, do_pos=true, do_lat=false)
imolecule_draw(h12o6)

### Geometry optimisation via ASE: preconditioning

In [ ]:
# ASE preconditioned minimiser
h12o6[:rattle](0.05)
h12o6[:set_calculator](fx)
#ucf = ase_const.UnitCellFilter(h12o6, [false,false,false,false,false,false])
minimiser = ase_precon.LBFGS(h12o6, precon=ase_precon.Exp(A=3.0))
minimiser[:run](fmax=1e-1)
imolecule_draw(h12o6)

## Random Search

In [ ]:

function random_h12o6()
    random_hex = make_h2o() # start with a single H2O
    for i=1:5               # add 5 more
        h2o = quippy.Atoms(n=3)
        reject = true
        while reject        # try to place each one NOT on top of another
            h2o = make_h2o()
            h2o[:rotate](quippy.random_unit_vector(), rand()*π)
            h2o[:set_positions](h2o[:get_positions]()+repeat(quippy.random_unit_vector()'*2.5, outer=[3,1]))
            reject = false
            for j=1:i       # check for Oxygens being too close
                if norm(h2o[:get_positions]()[1,:] - random_hex[:get_positions]()[(j-1)*3+1,:]) < 2.5
                    reject = true
                    break
                end
            end
        end  
        random_hex[:add_atoms](h2o)
    end
    return random_hex
end

In [ ]:
init = []
list = []
for i=1:50
    hex = random_h12o6()
    append!(init, [hex[:copy]()])
    res = optimize(tip3p.potential, (p,s)->s[:]=tip3p.gradient(p), vec(hex[:get_positions]()'),
                ftol=1e-2, grtol=1e-2)
    hex[:set_positions](reshape(res.minimum, (3,18))')
    append!(list, [hex])
    println("Random hexamer $i")
end

In [ ]:
list2=copy(list)
imolecule_draw(list2[5])


In [ ]:
function compute_h12o6_Rg(x)
    rO = reshape(x, (3,18))[:,1:3:18] # get Oxygen positions
    mr = mean(rO, 2)                  # compute the center of mass
    sqrt(mean(sum((rO-repeat(mr, inner=[1,6])).^2,1)))
end

In [ ]:
E = []; Rg = []
outfile = quippy.CInOutput("hexamers_random_search.xyz", quippy.OUTPUT)
for hex in list
    x = vec(hex[:get_positions]()')
    append!(Rg, [compute_h12o6_Rg(x)])
    append!(E, [tip3p.potential(x)])
    outfile[:write](hex)
end 
outfile[:close]()

In [ ]:
figure()
plot(Rg, E, "x")
xlabel("Radius of gyration"); ylabel("Energy"); title("Optimized random water hexamers")
close(gcf())

## MD via QUIP

In [ ]:

@pyimport quippy._dynamicalsystem as quippy_ds
outfile = quippy.CInOutput("traj.xyz", quippy.OUTPUT)
ds = quippy.DynamicalSystem(h12o6)
ds[:add_thermostat](quippy.THERMOSTAT_LANGEVIN, 300.0, tau=10.0)
@time traj = ds[:run](fx, dt=1.0, n_steps=5, summary_interval=0, save_interval=0, trajectory=outfile, write_interval=1)
outfile[:close]()